# Free Field Analysis Example

This example shows how to run OpenSees in DesignSafe from a jupyter notebook and how to postprocess the output results using python scripts, generate a LaTex report, and create interactive plots. 

A simple 1D free field analysis of a liquefiable soil layer is analyzed using OpenSees. An schematic of the soil profile in shown in the Figure. The soil profile consists of a 1 m dry crust, 3 m liquefiable layer, and 1 m of elastic base. The ground water table is at 2 m. An earthquake excitation is applied at the bottom of the soil column. A compliant rock is considered in the analysis. 

The results are presented in terms of:

a) Time history of acceleration at the surface and corresponding response spectra.

b) Profiles of maximum displacement, peak horizontal acceleration (PHA), maximum shear strain, and stress ratio

c) Stress strain plots for a point near the center of the liquefiable zone, and

d) Evolution of pore water pressure for a point near the center of the liquefiable zone. 

<img src = "schematic.png"  height="200" width="200" align = "center">

# Setup agave and start OpenSees job

### Setup job description

In [ ]:
from agavepy.agave import Agave
ag = Agave.restore()
app_id = "opensees-docker-2.5.0.6248u17"
storage_id = 'designsafe.storage.default'
input_dir = '/parduino/freeFieldEffectiveJupyter'
input_uri = 'agave://{}/{}'.format(storage_id, input_dir)
input_filename = 'N10_T3.tcl'
app = ag.apps.get(appId=app_id)
job_description = {
    "name": "opensees-ex",
    "appId" : app_id,
    #"executionSystem" : "designsafe.community.data-01",
    "batchQueue" : "debug",
    "nodeCount" : 1,
    "processorsPerNode" : 1,
    "memoryPerNode" : "1",
    "maxRunTime" : "00:30:00",
    "archive" : True,
    "inputs" : {
       "inputDirectory" :  input_uri 
    },
    "parameters" : {
      "inputScript" :  input_filename
    }
}

### Run job

In [ ]:
job = ag.jobs.submit(body=job_description)
from agavepy.async import AgaveAsyncResponse
asrp = AgaveAsyncResponse(ag, job)
asrp.result()

# Postprocess Results

### Go to archived folder

In [ ]:
jobinfo = ag.jobs.get(jobId=job.id)

In [ ]:
jobinfo.archivePath

In [ ]:
user = jobinfo.archivePath.split('/', 1)[0]

In [ ]:
import os

In [ ]:
%cd ..

In [ ]:
os.chdir(jobinfo.archivePath.replace(user,'/home/jupyter/MyData'))

In [ ]:
if not os.path.exists('freeFieldEffectiveJupyter'):
    os.makedirs('freeFieldEffectiveJupyter')

In [ ]:
os.chdir('freeFieldEffectiveJupyter')

### Import python libraries

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

### Plot acceleration time history

Plot acceleration time hisotory and response spectra on log-linear scale

In [ ]:
from plotAcc import plot_acc
plot_acc()

### Plot profiles

Plot profiles of max displacement, PGA, max shear strain, stress ratio and plot stress strain near the center of liquefiable layer 

In [ ]:
from plotProfile import plot_profile
plot_profile()

### Plot excess pore water pressure

In [ ]:
from plotPorepressure import plot_porepressure
plot_porepressure()

# Generate LaTeX Report 

In [ ]:
os.system('/usr/bin/pdflatex -interaction nonstopmode ShortReport.tex')

In [ ]:
class PDF(object):
  def __init__(self, pdf, size=(200,200)):
    self.pdf = pdf
    self.size = size

  def _repr_html_(self):
    return '<iframe src={0} width={1[0]} height={1[1]}></iframe>'.format(self.pdf, self.size)

  def _repr_latex_(self):
    return r'\includegraphics[width=1.0\textwidth]{{{0}}}'.format(self.pdf)

In [ ]:
pdf_fn = jobinfo.archivePath.replace(user, '/user/' + user + '/files/MyData')
pdf_fn += '/freeFieldEffectiveJupyter/ShortReport.pdf'
print pdf_fn

In [ ]:
PDF(pdf_fn , (750,600))

# Create Interactive Plots

### Pore water pressure

In [ ]:
from interactiveplot import createpwpplot, createDispplot

In [ ]:
createpwpplot()

### Displacement

In [ ]:
createDispplot()